In [49]:
import pandas as pd
import xml.etree.ElementTree as ET

# Especifica la ruta al archivo Excel
archivo_excel = 'extract_glosa.xlsx'

# Lee el archivo Excel en un DataFrame de Pandas
df = pd.read_excel(archivo_excel)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   id            934 non-null    int64         
 1   rutemisor     934 non-null    object        
 2   folio         934 non-null    int64         
 3   date          934 non-null    datetime64[ns]
 4   xml_dte       934 non-null    object        
 5   convert_from  934 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 43.9+ KB


In [40]:
# SinDscItem = pd.DataFrame()
# df_con = pd.DataFrame()
filas_sin = []
filas_con = []

i = 0

for xml in df['convert_from'].head(20):

    root = ET.fromstring(xml)

    # Utiliza el namespace de XML
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    rut_emisor = root.find('.//default:RutEmisor', namespace).text

    razon_social = root.find('.//{http://www.sii.cl/SiiDte}RznSoc').text

    NmbItem = root.find('.//{http://www.sii.cl/SiiDte}NmbItem').text

    # a veces DscItem no existe
    DscItem = root.find('.//{http://www.sii.cl/SiiDte}DscItem')

    if DscItem is not None: # Crea nuevas filas para el df con DscItem
        nuevaFila = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            "NmbItem": NmbItem,
            "DscItem": root.find('.//{http://www.sii.cl/SiiDte}DscItem').text
        }
        filas_con.append(nuevaFila)
    else: # Crea nuevas filas para el df sin DscItem
        nuevaFila = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            "NmbItem": NmbItem,
        }
        filas_sin.append(nuevaFila)

# agregar las nuevas filas a los distintos df
df_con = pd.DataFrame(filas_con)
df_sin = pd.DataFrame(filas_sin)

print(f'filas totales: {df_con.shape[0]+df_sin.shape[0]}')

filas totales: 20


In [57]:
# SinDscItem = pd.DataFrame()
# df_con = pd.DataFrame()
filas = []


i = 0

for xml in df['convert_from'].head(20):

    root = ET.fromstring(xml)

    # Utiliza el namespace de XML
    namespace = {'default': 'http://www.sii.cl/SiiDte'}

    rut_emisor = root.find('.//default:RutEmisor', namespace).text

    razon_social = root.find('.//{http://www.sii.cl/SiiDte}RznSoc').text

    NmbItem = root.find('.//{http://www.sii.cl/SiiDte}NmbItem').text

    # a veces DscItem no existe
    DscItem = root.find('.//{http://www.sii.cl/SiiDte}DscItem')

    if DscItem is not None: # Crea nuevas filas para el df con DscItem
        nuevaFila = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            # "NmbItem": NmbItem,
            # "DscItem": root.find('.//{http://www.sii.cl/SiiDte}DscItem').text,
            "Glosa": f'{NmbItem} {DscItem.text}'
        }
        filas.append(nuevaFila)
    else: # Crea nuevas filas para el df sin DscItem
        nuevaFila = {
            "rut_emisor": rut_emisor,
            "razon_social": razon_social,
            # "NmbItem": NmbItem,
            "Glosa": NmbItem,
        }
        filas.append(nuevaFila)

# agregar las nuevas filas a los distintos df
df_final = pd.DataFrame(filas)



In [60]:
# Convierte el DataFrame a CSV
csv_data = df_final.to_csv('csv_data',index=False)